Set colab to access the images in Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Image sample folder setting

In [ ]:
import shutil


# Define the source and destination folders
source_folder = '/content/drive/My Drive/RealData_resized'
destination_folder = '/content/drive/My Drive/Sample/Real'

# Ensure the destination folder exists
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# Get the list of image files in the source folder
image_files = [f for f in os.listdir(source_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]

# Specify the number of images you want to copy
number_of_images_to_copy = 600

# Copy the specified number of images to the destination folder
for i, image_file in enumerate(image_files):
    if i < number_of_images_to_copy:
        source_path = os.path.join(source_folder, image_file)
        destination_path = os.path.join(destination_folder, image_file)
        shutil.copy(source_path, destination_path)
    else:
        break

print(f'Copied {number_of_images_to_copy} images to {destination_folder}')


NameError: name 'os' is not defined

In [ ]:
num_files = len([f for f in os.listdir(destination_folder) if os.path.isfile(os.path.join(destination_folder, f))])

print(f'There are {num_files} files in the folder.')

## Model

Import the required libraries

In [ ]:
!pip install tensorflow

In [3]:
import os   # To navigate into the drive folders and access the images
import matplotlib.pyplot as plt   # To make plots
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator  # To process images in batches and apply transformations
from keras.applications import VGG19   # Import the VGG19 model
from keras.models import Sequential    # Allow us to add another layer to the pretrained model that makes the classification
from keras.layers import Flatten, Dense, Dropout   # Flatten the output layer to make it able for input in the Dense layer
from keras.optimizers import Adam    # Import the Adam optimizer


Defining paths to our images

In [4]:
# Define the base directory where your 'sample' folder is located
base_dir = '/content/drive/My Drive/Sample'

# Define the paths to the training, validation, and testing directories
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

Data normalization and ImageDataGenerator instances

In [5]:
# Rescale the pixel values from [0, 255] to [0, 1] for normalization

train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen  = ImageDataGenerator(rescale=1./255)
test_datagen  = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224,224),   # This is the input size required for the VGG19 model
    batch_size=20,  # It will be iterating from batches of 20 images
    class_mode= 'binary'  # Label the images in a binary way (Fake or Real)
)

Validation_generator = validation_datagen .flow_from_directory(
    validation_dir,
    target_size=(224,224),   # This is the input size required for the VGG19 model
    batch_size=20,  # It will be iterating from batches of 20 images
    class_mode= 'binary'  # Label the images in a binary way (Fake or Real)
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224,224),   # This is the input size required for the VGG19 model
    batch_size=20,  # It will be iterating from batches of 20 images
    class_mode= 'binary',  # Label the images in a binary way (Fake or Real)
    shuffle=False)  # No need to shuffle the test data

Found 800 images belonging to 2 classes.
Found 200 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [6]:
# Load the VGG19 model pre-trained on ImageNet, excluding its top (fully connected) layers
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the base model to prevent them from being updated during training
for layer in base_model.layers:
    layer.trainable = False

# Create a new Sequential model and add the VGG19 base model without dropout
model = Sequential([
    base_model,
    Flatten(),  # Flatten the output of the base model to a 1D vector
    Dense(256, activation='relu'),  # Add a fully connected layer with 256 units and ReLU activation
    Dropout(0.4),  # Add dropout for regularization (reduce overfitting)
    Dense(1, activation='sigmoid')  # Add the output layer with sigmoid activation for binary classification
])


80134624/80134624 [==============================] - 1s 0us/step


In [7]:
# Model compilation with Adam activation and binary_crossentropy loss function, good for binary classifications
# Metrics also included

model.compile(optimizer=Adam(learning_rate= 1e-4),
               loss='binary_crossentropy',
               metrics=['accuracy', 'Precision', 'Recall'])

First model training

In [8]:
# Model training
num_epochs = 10

history = model.fit(
    train_generator,
    steps_per_epoch= 40,   # Batch size is 20, 20 times 40 is 800, the size of the training data
    epochs=num_epochs,
    validation_data=Validation_generator,
    validation_steps = 5
)

Epoch 1/10
40/40 [==============================] - 200s 5s/step - loss: 0.7005 - accuracy: 0.6313 - precision: 0.6290 - recall: 0.6400 - val_loss: 0.4985 - val_accuracy: 0.7300 - val_precision: 0.7442 - val_recall: 0.6667
Epoch 2/10
40/40 [==============================] - 37s 945ms/step - loss: 0.4283 - accuracy: 0.8163 - precision: 0.8286 - recall: 0.7975 - val_loss: 0.4505 - val_accuracy: 0.8200 - val_precision: 0.7500 - val_recall: 0.9130
Epoch 3/10
40/40 [==============================] - 27s 683ms/step - loss: 0.3217 - accuracy: 0.8625 - precision: 0.8756 - recall: 0.8450 - val_loss: 0.4446 - val_accuracy: 0.7900 - val_precision: 0.7333 - val_recall: 0.8980
Epoch 4/10
40/40 [==============================] - 23s 568ms/step - loss: 0.2740 - accuracy: 0.8963 - precision: 0.8972 - recall: 0.8950 - val_loss: 0.3783 - val_accuracy: 0.8500 - val_precision: 0.8511 - val_recall: 0.8333
Epoch 5/10
40/40 [==============================] - 21s 521ms/step - loss: 0.1984 - accuracy: 0.9350 -

In [10]:
# Evaluate the model's performance on the test set
test_loss, test_accuracy, test_precision, test_recall = model.evaluate(test_generator)
print('Test accuracy:', test_accuracy)

10/10 [==============================] - 4s 367ms/step - loss: 0.3322 - accuracy: 0.8350 - precision: 0.8764 - recall: 0.7800
Test accuracy: 0.8349999785423279


Based on the results, we se that 8 epochs is the optimal number of epochs to get the best results, we are making a second model increasing the dropout and setting just 8 epochs. Remember that it is working with sample data

In [14]:
# Freeze the layers of the base model to prevent them from being updated during training
for layer in base_model.layers:
    layer.trainable = False


model = Sequential([
    base_model,
    Flatten(),  # Flatten the output of the base model to a 1D vector
    Dense(256, activation='relu'),  # Add a fully connected layer with 256 units and ReLU activation
    Dropout(0.5),  # Add dropout for regularization (reduce overfitting)
    Dense(1, activation='sigmoid')  # Add the output layer with sigmoid activation for binary classification
])

# Model compilation with Adam activation and binary_crossentropy loss function, good for binary classifications
# Metrics also included

model.compile(optimizer=Adam(learning_rate= 1e-4),
               loss='binary_crossentropy',
               metrics=['accuracy', 'Precision', 'Recall'])

# Model training
num_epochs = 8

history = model.fit(
    train_generator,
    steps_per_epoch= 40,   # Batch size is 20, 20 times 40 is 800, the size of the training data
    epochs=num_epochs,
    validation_data=Validation_generator,
    validation_steps = 5
)

Epoch 1/8
40/40 [==============================] - 22s 504ms/step - loss: 0.6923 - accuracy: 0.6250 - precision: 0.6269 - recall: 0.6175 - val_loss: 0.5549 - val_accuracy: 0.7200 - val_precision: 0.6885 - val_recall: 0.8235
Epoch 2/8
40/40 [==============================] - 20s 498ms/step - loss: 0.4644 - accuracy: 0.7800 - precision: 0.7800 - recall: 0.7800 - val_loss: 0.4893 - val_accuracy: 0.7700 - val_precision: 0.7213 - val_recall: 0.8800
Epoch 3/8
40/40 [==============================] - 19s 479ms/step - loss: 0.3686 - accuracy: 0.8487 - precision: 0.8642 - recall: 0.8275 - val_loss: 0.4695 - val_accuracy: 0.8000 - val_precision: 0.7377 - val_recall: 0.9184
Epoch 4/8
40/40 [==============================] - 22s 559ms/step - loss: 0.2799 - accuracy: 0.8950 - precision: 0.8990 - recall: 0.8900 - val_loss: 0.4053 - val_accuracy: 0.8300 - val_precision: 0.8077 - val_recall: 0.8571
Epoch 5/8
40/40 [==============================] - 19s 484ms/step - loss: 0.2281 - accuracy: 0.9262 - pr

In [15]:
# Evaluate the model's performance on the test set
test_loss, test_accuracy, test_precision, test_recall = model.evaluate(test_generator)
print('Test accuracy:', test_accuracy)

10/10 [==============================] - 5s 416ms/step - loss: 0.3643 - accuracy: 0.8300 - precision: 0.8235 - recall: 0.8400
Test accuracy: 0.8299999833106995
